<a href="https://colab.research.google.com/github/bhavi289/DL-GANs/blob/master/DL_Assign_02_Q1_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

!pip install -q pyyaml
!pip install gdown==3.6.0

In [0]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
import os
from sklearn.model_selection import train_test_split
from google.colab import drive
import glob
import imageio
%matplotlib inline 
import matplotlib.pyplot as plt 
import numpy as np
import os 
import PIL
import tensorflow.keras.layers as layers
import time
from IPython import display
from tensorflow.python.client import device_lib
from PIL import Image

drive.mount('/content/drive')

In [0]:
'''
  Load Dataset
'''
dataset_path = 'drive/My Drive/Semester 6/DL/Assignment 2/faces94/'
flag = 0
X = []
for root, directories, files in os.walk(dataset_path):
    
  for file in files:
    try:
        file_path = root + '/' + file
        extension_list = ['jpg', 'jpeg', 'png', 'svg']
        if (file_path.split('.')[-1] in extension_list):
          X.append(file_path)
    except Exception as e:
        print (e)   
        pass

print("no. of images = {}".format(len(X)))

In [0]:
def load_image( infilename ) :
    img = PIL.Image.open( infilename )
    img = img.resize((64,64))
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

In [0]:
plt.imshow(load_image(X[3000]), interpolation='nearest')
plt.show()

In [0]:
test = load_image(dataset_path + "male/gjnorm/gjnorm.6.jpg")

In [0]:
print("Test Shape = {}".format(str(test.shape)))

In [0]:
plt.imshow(test, interpolation='nearest')
plt.show()

In [0]:
'''
  Making Minibatches
'''
BUFFER_SIZE = 2944
BATCH_SIZE = 32

print("Number Of Batches = {}".format(BUFFER_SIZE//BATCH_SIZE))

Number Of Batches = 92


In [0]:
train_data = np.array(X)
np.random.shuffle(train_data)

train_data = np.split(train_data[:BUFFER_SIZE], BUFFER_SIZE//BATCH_SIZE)

print("Number of Bactches = {}".format(len(train_data)))
print("Batch Size = {}".format(len(train_data[0])))

Number of Bactches = 92
Batch Size = 32


In [0]:
def make_generator_model():
  """
    Implements the forward propagation for the model:
      SEQUENTIAL -> DENSE -> NORM -> RELU -> CONV2D -> NORM -> RELU -> CONV2D -> NORM -> RELU -> CONV2D -> NORM -> RELU -> CONV2D
    
    Arguments:
      None

    Returns:
      Generator Model
  """
  model = tf.keras.Sequential()
  
  model.add(layers.Dense(4*4*1024, use_bias = False, input_shape = (100,)))
  
  model.add(layers.BatchNormalization())
  
  model.add(layers.LeakyReLU())
  
  model.add(layers.Reshape(( 4, 4, 1024)))
  
  model.add(layers.Conv2DTranspose(512, (5, 5), strides = (2,2), padding = "same", use_bias = False))
  
  model.add(layers.BatchNormalization())
  
  model.add(layers.LeakyReLU())
  
  model.add(layers.Conv2DTranspose(256, (5,5), strides = (2,2), padding = "same", use_bias = False))
  
  model.add(layers.BatchNormalization())
  
  model.add(layers.LeakyReLU())
  
  model.add(layers.Conv2DTranspose(128, (5,5), strides = (2,2), padding = "same", use_bias = False))
  
  model.add(layers.BatchNormalization())
  
  model.add(layers.LeakyReLU())
  
  model.add(layers.Conv2DTranspose(3, (5,5), strides = (2,2), padding = "same", use_bias = False, activation = "tanh"))
  
  print("Model Shape = {}".format(model.output_shape))
  
  return model

In [0]:
generator = make_generator_model()

noise = tf.random.normal([1,100])
generated_image = generator(noise, training = False)

print(generated_image.shape)

Model Shape = (None, 64, 64, 3)
(1, 64, 64, 3)


In [0]:
def make_discriminator_model():
    """
      Implements the forward propagation for the model:
        SEQUENTIAL -> CONV2D -> RELU -> DROPOUT -> CONV2D -> RELU -> DROPOUT -> FLATTEN -> DENSE

      Arguments:
        None

      Returns:
        Discriminator Model
    """
    model = tf.keras.Sequential()
    
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[64, 64, 3]))
    
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dropout(0.3))
      
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Flatten())
    
    model.add(layers.Dense(1))
     
    return model

In [0]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print(decision)

tf.Tensor([[-4.633789e-05]], shape=(1, 1), dtype=float32)


In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits = True)

In [0]:
def discriminator_loss(real_output, fake_output):
  """
    Return Discriminator Loss
    
    Arguments:
      real_output - Real Output
      fake_output - Fake Output

    Returns:
      Discriminator Loss
  """
  real_loss = cross_entropy(tf.ones_like(real_output), real_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  total_loss = real_loss + fake_loss
  
  return total_loss

In [0]:
def generator_loss(fake_output):
  """
    Return Generator Loss
    
    Arguments:
      fake_output - Fake Output

    Returns:
      Generator Loss
  """
  return cross_entropy(tf.ones_like(fake_output), fake_output)

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [0]:
"""
  Checkpoint directory
"""
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer = generator_optimizer, discriminator_optimizer = discriminator_optimizer, generator = generator, discriminator = discriminator)

In [0]:
"""
  Saving Generator Weights
"""
generator.save_weights('./training_checkpoints/md')

In [0]:
generator1  = make_generator_model()
generator1.load_weights('./training_checkpoints/md')

Model Shape = (None, 64, 64, 3)


In [0]:
noise = tf.random.normal([1,100])
generated_image = generator1(noise, training = False)

In [0]:
EPOCHS = 50
noise_dim = 100
num_examples_to_generate = 64

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [0]:
def load_image( infilename ) :
    img = PIL.Image.open( infilename )
    img = img.resize((64,64))
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

In [0]:
def train_step(images):
    """
      The training loop begins with generator receiving a random seed as input. 
      That seed is used to produce an image. The discriminator is then used to classify real images (drawn from the training set) and fakes images (produced by the generator). 
      The loss is calculated for each of these models, and the gradients are used to update the generator and discriminator.
      
      Args:
        Images
      
      Output:
        Generator Loss, Discriminator Loss
    """
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    new_images = []
    i = 0 
    for file_name in images:
        i+=1
        new_pic = load_image(file_name)
        new_images.append(new_pic)
    
    images = np.array(new_images)
    images = images.reshape(images.shape[0], 64, 64, 3).astype('float32')
    images = (images - 127.5) / 127.5

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    images = None
    
    return gen_loss, disc_loss

In [0]:
generator_loss_epoch = []
discriminator_loss_epoch = []

In [0]:
def train(dataset, epochs):  
  """
    Training of Model
  """
  
  for epoch in range(epochs):
    start = time.time()
    batch_no = 1
    for image_batch in dataset:
      batch_no += 1
      g_loss, d_loss = train_step(image_batch)
      
    generator_loss_epoch.append(g_loss)
    discriminator_loss_epoch.append(d_loss)
    
    print("Epoch = {}".format(epoch+1))
    print("\tGenerator Loss = {}".format(g_loss))
    print("\tDiscriminator Loss = {}".format(d_loss))

    generate_and_save_images(generator, epoch + 1, seed)
    
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)
    
  generate_and_save_images(generator, epochs, seed)

In [0]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training=False)

  result = predictions[0]
  plt.imshow(result)

In [0]:
def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))
  
  for i in range(predictions.shape[0]):
      plt.subplot(8, 8, i+1)
      plt.axis('off')

In [0]:
%%time
train(train_data, 50)

In [0]:
generate_and_save_images(generator, 50, seed)

In [0]:
generator_loss_epoch[0]